In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('nlp_prac').getOrCreate()

24/11/03 04:18:12 WARN Utils: Your hostname, AlienEE resolves to a loopback address: 127.0.1.1; using 10.3.0.66 instead (on interface wlp4s0)
24/11/03 04:18:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/03 04:18:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/11/03 04:18:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
data = spark.read.csv('smsspamcollection/SMSSpamCollection', inferSchema=True, sep='\t')

In [6]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [7]:
data.show()

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
|  ham|I'm gonna be home...|
| spam|SIX chances to wi...|
| spam|URGENT! You have ...|
|  ham|I've been searchi...|
|  ham|I HAVE A DATE ON ...|
| spam|XXXMobileMovieClu...|
|  ham|Oh k...i'm watchi...|
|  ham|Eh u remember how...|
|  ham|Fine if thats th...|
| spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows



In [8]:
from pyspark.sql.functions import length

In [9]:
data = data.withColumn('length', length(data['text']))

In [10]:
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [13]:
data.groupBy('class').mean().show()

+-----+-----------------+
|class|      avg(length)|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



In [14]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer

In [16]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

In [17]:
from pyspark.ml.feature import VectorAssembler

In [18]:
clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [19]:
from pyspark.ml.classification import  NaiveBayes

In [20]:
nb = NaiveBayes()

In [21]:
from pyspark.ml import Pipeline

In [22]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up])

In [23]:
cleaner = data_prep_pipe.fit(data)

In [24]:
clean_data = cleaner.transform(data)

In [25]:
clean_data.columns

['class',
 'text',
 'length',
 'label',
 'token_text',
 'stop_token',
 'c_vec',
 'tf_idf',
 'features']

In [26]:
clean_data = clean_data.select('label', 'features')

In [27]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(13424,[7,11,31,6...|
|  0.0|(13424,[0,24,301,...|
|  1.0|(13424,[2,13,19,3...|
|  0.0|(13424,[0,70,80,1...|
|  0.0|(13424,[36,134,31...|
|  1.0|(13424,[10,60,140...|
|  0.0|(13424,[10,53,102...|
|  0.0|(13424,[127,185,4...|
|  1.0|(13424,[1,47,121,...|
|  1.0|(13424,[0,1,13,27...|
|  0.0|(13424,[18,43,117...|
|  1.0|(13424,[8,16,37,8...|
|  1.0|(13424,[13,30,47,...|
|  0.0|(13424,[39,95,221...|
|  0.0|(13424,[555,1797,...|
|  1.0|(13424,[30,109,11...|
|  0.0|(13424,[82,214,44...|
|  0.0|(13424,[0,2,49,13...|
|  0.0|(13424,[0,74,105,...|
|  1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows



In [28]:
train_data, test_data = clean_data.randomSplit([0.7, 0.3])

In [29]:
spam_detector = nb.fit(train_data)

24/11/03 04:29:10 WARN DAGScheduler: Broadcasting large task binary with size 1166.8 KiB
24/11/03 04:29:11 WARN DAGScheduler: Broadcasting large task binary with size 1146.6 KiB


In [30]:
data.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)



In [31]:
test_results = spam_detector.transform(test_data)

In [32]:
test_results.show()

24/11/03 04:29:43 WARN DAGScheduler: Broadcasting large task binary with size 1371.8 KiB


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(13424,[0,1,2,13,...|[-605.46911521534...|[1.0,9.8311303180...|       0.0|
|  0.0|(13424,[0,1,5,20,...|[-799.10149236035...|[1.0,2.4342763238...|       0.0|
|  0.0|(13424,[0,1,7,15,...|[-659.28962025634...|[1.0,2.0591685780...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-562.04297349347...|[1.0,1.8140855189...|       0.0|
|  0.0|(13424,[0,1,9,14,...|[-562.04297349347...|[1.0,1.8140855189...|       0.0|
|  0.0|(13424,[0,1,14,18...|[-1359.6805367453...|[1.0,1.4545377827...|       0.0|
|  0.0|(13424,[0,1,14,31...|[-218.18944113021...|[1.0,6.2011442781...|       0.0|
|  0.0|(13424,[0,1,14,79...|[-691.91932460178...|[1.0,8.7473060842...|       0.0|
|  0.0|(13424,[0,1,21,27...|[-1009.2575590336...|[1.0,2.7528424522...|       0.0|
|  0.0|(13424,[0

In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [34]:
acc_eval = MulticlassClassificationEvaluator()

In [35]:
acc = acc_eval.evaluate(test_results)

24/11/03 04:30:20 WARN DAGScheduler: Broadcasting large task binary with size 1376.9 KiB


In [36]:
print(f"Accuracy of NB model {acc}")

Accuracy of NB model 0.9353339154488983
